In [7]:
import cv2 as cv
import numpy as np
#from skimage import data, filters

In [8]:
filename=r"C:\Users\54922\GITHUB\vision_computadora_I\TP_Entrega_vision_por_computadora_1\vtest.avi"
cap = cv.VideoCapture(filename)
if not cap.isOpened:
    print('Falla al abrir el archivo: ' + filename)
    exit(0)

In [9]:
#definimos los metodos de substracción de fondo mediante Gaussiano y Mediana 
backSubGauss = cv.createBackgroundSubtractorMOG2()

def backSubMedian(frames):
    backSubMedian = np.median(frames, axis=0).astype(dtype=np.uint8) 
    return backSubMedian



### Extracción de fondo usando la mascara de mediana

In [10]:
# Se seleccionan 24 frame random
frameIds = cap.get(cv.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=24)
frames = []
for fid in frameIds:
    cap.set(cv.CAP_PROP_POS_FRAMES, fid)
    ret, frame = cap.read()
    frames.append(frame)

#Se calcula mediana de frames
fmedian = backSubMedian(frames)
#Se muestra el frame calculado con mediana
cv.imshow('Median frame', fmedian)
cv.waitKey(0)

"\ncv.imshow('Median frame', fmedian)\ncv.waitKey(0)\n"

In [11]:
#Aca conviene que el video esté dentro del proyecto. sino tira un error vimculado a path

# Se resetea cantidad de frames a 0
cap.set(cv.CAP_PROP_POS_FRAMES, 0)
# se convierte el fondo a escala de grises
grayMedianFrame = cv.cvtColor(fmedian, cv.COLOR_BGR2GRAY)

"\nwhile True:\n# Leemos frames\n\tret, frame = cap.read()\n# Convertimos a escala de grises\n\tframe = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)\n# Se calcula la diferencia absoluta entre el frame y mediana\n\tdframe = cv.absdiff(frame, grayMedianFrame)\n\tth, dframe = cv.threshold(dframe, 30, 255, cv.THRESH_BINARY)\n# Display image\n\tcv.imshow('frame', dframe)\n\tkeyboard=cv.waitKey(30)\n\tif keyboard == 'q' or keyboard == 27:\n\t\tbreak\n\n\t\ncv.destroyAllWindows()\ncap.release()\n"

In [12]:
# Corremos la sustraccion
#------------------------
while True:
    # Leemos un frame
    ret, frame = cap.read()
    if frame is None:
        break
    
    # Aplicamos la sustracción al frame leído
    #----------------------------------------
    # Cada frame se utiliza tanto para calcular la máscara de primer plano como para actualizar el fondo.
    # Si se desea cambiar la tasa de aprendizaje utilizada para actualizar el modelo de fondo, es posible
    # establecer una tasa de aprendizaje específica pasando un parámetro al método apply.

    #######    GAUSSIAN  #############
    fgMask = backSubGauss.apply(frame)

    #######    MEDIAN   ##############
    frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Se calcula la diferencia absoluta entre el frame y mediana
    dframe = cv.absdiff(frame, grayMedianFrame)
    th, fmMask = cv.threshold(dframe, 30, 255, cv.THRESH_BINARY)
    ##################################
    
    # Escribimos sobre la imagen el número de frame procesado
    cv.rectangle(frame, (10, 2), (100,20), (255,255,255), -1)
    cv.putText(frame, str(cap.get(cv.CAP_PROP_POS_FRAMES)), (15, 15),
               cv.FONT_HERSHEY_SIMPLEX, 0.5 , (0,0,0))
    
    # mostramos frame original e imagen binaria background/foreground
    cv.imshow('Frame', frame)
    cv.imshow('FG Mask', fgMask)
    cv.imshow('frame', fmMask)



    # Corresmos hasta que termine o apriete escape
    keyboard = cv.waitKey(30)
    if keyboard == 'q' or keyboard == 27:
        break

cv.destroyAllWindows()
cap.release()

### * Se observa menor ruido y mejor manejo de sombras en el caso de extracción de fondo utilizando mediana. 